In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

DRL:  
    - 全称(Deep Reinforcement Learning) 深度强化学习  
    - 强调基于环境行动，以取得最大化预期利益  
    - alphaGO即是深度强化学习的典型应用  